<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #12: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Oct 31st, 2021</h5>

# Set up and requirements

### Import Python packages

In [2]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.tutorials_utils as t_utils
import utils.t12_utils as t12
import utils.zooniverse_utils as zoo

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [3]:
project = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

### Initiate SQL database and populate sites, movies and species

In [4]:
# Initiate the SQL database 
%run -i "../db_starter/starter.py" --project_name $project.value

Updated sites
Updated movies
Updated species


### Retrieve Zooniverse information

In [5]:
# Save your Zooniverse user name and password.
zoo_user, zoo_pass = zoo.zoo_credentials()

Enter your Zooniverse user········
Enter your Zooniverse password········


In [7]:
# Specify the Zooniverse information required throughout the tutorial
zoo_info = ["subjects", "workflows", "classifications"]

# Retrieve and store the Zooniverse information required throughout the tutorial in a dictionary
zoo_info_dict = zoo.retrieve_zoo_info(zoo_user, zoo_pass, project.value, zoo_info)

Connecting to the Zooniverse project
Retrieving subjects from Zooniverse
subjects were retrieved successfully
Retrieving workflows from Zooniverse
workflows were retrieved successfully
Retrieving classifications from Zooniverse
classifications were retrieved successfully


In [8]:
# Populate the sql with subjects uploaded to Zooniverse
zoo.populate_subjects(zoo_info_dict["subjects"], project.value)

Updated subjects
The database has a total of 2342 frame subjects and 7362 clip subjects have been updated


### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [9]:
# Display a selectable list of workflow names and the type of subject
workflows_df = zoo_info_dict["workflows"]
workflow_name, subj_type = t_utils.choose_single_workflow(workflows_df)

Dropdown(description='Workflow name:', options=('Species identification', 'test ', 'Species location (intermed…

Dropdown(description='Subject type:', index=1, options=('frame', 'clip'), value='clip')

In [10]:
# Selects the workflow id based on the workflow name
workflow_id = workflows_df[workflows_df.display_name==workflow_name.value].workflow_id.unique()[0]

# Display a selectable list of versions of the workflow of interest
workflow_version = t12.choose_w_version(workflows_df, workflow_id)

Dropdown(description='Minimum workflow version:', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, …

In [11]:
# Retrieve classifications from the workflow of interest
class_df = t12.get_classifications(workflow_id = workflow_id,
                                   workflow_version = workflow_version.value, 
                                   subj_type = subj_type.value, 
                                   class_df = zoo_info_dict["classifications"],
                                   project_name = project.value)

There are 2747 classifications out of 54324 missing subject info. Maybe the subjects have been removed from Zooniverse?
Zooniverse classifications have been retrieved


### Step 2: Aggregate classifications received on the workflow of interest

In [12]:
# Specify the agreement threshold required among cit scientists
agg_params = t12.choose_agg_parameters(subj_type.value)

FloatSlider(value=0.8, continuous_update=False, description='Aggregation threshold:', max=1.0, readout_format=…

IntSlider(value=3, continuous_update=False, description='Min numbers of users:', max=15, min=1, style=SliderSt…

In [13]:
agg_class_df, raw_class_df = t12.aggregrate_classifications(class_df, subj_type.value, project.value, agg_params)

Aggregrating the classifications
1878 classifications aggregated out of 5202 unique subjects available


### Step 3: Summarise the number of classifications based on the agreement specified

In [14]:
agg_class_df.groupby("label")["subject_ids"].agg("count")

,subject_ids
label,


### Step 4: Display the aggregated classifications in a table

In [15]:
# Display the dataframe into a table
if subj_type.value == "clip":
    a = agg_class_df[["subject_ids","label","how_many","first_seen"]]
else:
    a = agg_class_df
a

,subject_ids,label,how_many,first_seen


### Step 5: Use the subject explorer widget to visualise subjects and their aggregated classifications

In [16]:
# Launch the subject viewer
t12.launch_viewer(agg_class_df, subj_type.value)

Combobox(value='', description='Subject id:', ensure_option=True, options=('38166820', '38166821', '38166823',…

Output()

### Step 6: Use the subject explorer widget to get more information about specific subjects and their "raw" classifications

In [17]:
# Launch the classifications_per_subject explorer
t12.explore_classifications_per_subject(raw_class_df, subj_type.value)

Combobox(value='', description='Subject id:', ensure_option=True, options=('38166866', '38166895', '38166843',…

Output()

In [ ]:
# END